# 8-Puzzle Solver

In [46]:
import heapq
import time
import psutil
import os

## get_ram_usage()
Função para obter o total de memória ram usada (em MB) na execução dos algoritmos

In [47]:
def get_ram_usage():
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / 1024 / 1024  # MB

## rebuild_path(start, finish)
Função para reconstruir o caminho feito entre dois estados do puzzle.

Seu propósito é puramente por fins de economia de memória. Com ela, é possível guardar apenas as transições de estado entre os nós da árvore e reconstruir o caminho correto no final, ao invés de guardar todos os caminhos explorados.
### Exemplo de uso
Seja o estado inicial (1, 2, 3, 4, 5, 6, 7, 0, 8) e o estado final (1, 2, 3, 4, 5, 6, 7, 8, 0):
Nessa instância, a função retorna "Right", já que o 0 foi trocado de lugar com o número 8 (e o número 8 por sua vez estava à direita do 0 no puzzle)

In [48]:
def rebuild_path(start, finish):
    i = start.index("9")
    if i > 2:
        cpy = list(start)
        cpy[i], cpy[i - 3] = cpy[i - 3], cpy[i]
        if "".join(cpy) == finish:
            return "Up"
    if i < 6:
        cpy = list(start)
        cpy[i], cpy[i + 3] = cpy[i + 3], cpy[i]
        if "".join(cpy) == finish:
            return "Down"
    if i % 3 != 0:
        cpy = list(start)
        cpy[i], cpy[i - 1] = cpy[i - 1], cpy[i]
        if "".join(cpy) == finish:
            return "Left"
    if i % 3 != 2:
        cpy = list(start)
        cpy[i], cpy[i + 1] = cpy[i + 1], cpy[i]
        if "".join(cpy) == finish:
            return "Right"

## manhattan(current, goal="123456789"):
Função para calcular o quão distante cada peça está do estado final utilizando a distância Manhattan

In [49]:
def manhattan(current, goal="123456789"):
    dist = 0
    for i, val in enumerate(current):
        if val == '9':
          continue
        target = goal.index(val)
        dist += abs(target // 3 - i // 3) + abs(target % 3 - i % 3)
    return dist

## def get_next(a, b, current, tree, queue, cost, depth):
Função para obter o próximo estado do puzzle. Primeiro é feita a troca do 0 com o número adjacente (cima, baixo, esquerda ou direita). Depois, é verificado se o estado já foi visitado. Se não, ele é adicionado à fila e o nó é adicionado à árvore. O algoritmo A* incluí uma verificação para não incluir caminhos com custo muito elevados

In [50]:
def get_next_astar(a, b, current, tree, queue, cost, depth):
    l = list(current)
    l[a], l[b] = l[b], l[a]
    target = int(''.join(l))
    if target not in tree or depth + 1 < cost.get(target, float('inf')):
        cost[target] = depth + 1
        heapq.heappush(queue, (cost[target] + manhattan(str(target)), depth + 1, target))
        tree[target] = current

def get_next_gulosa(a, b, current, tree, queue, depth):
    l = list(current)
    l[a], l[b] = l[b], l[a]
    target = int(''.join(l))
    if target not in tree:
        heapq.heappush(queue, (manhattan(str(target)), depth + 1, target))
        tree[target] = current

## Astar(puzzle)  
### Algoritmo A*  
A função resolve o puzzle usando o algoritmo A*, que é uma busca heurística. Assim como nos algoritmos feitos anteriormente, antes de iniciar, são feitas otimizações:
- A tupla inicial é transformada em uma string e depois em inteiro, substituindo o `0` por `9`, para economizar memória e evitar problemas de conversão como no caso `012345678` virando `12345678`.

O A* usa uma fila de prioridade (`heapq`) para explorar os nós com menor custo estimado primeiro. Cada entrada da fila é uma tupla `(f, depth, puzzle)` onde `f = custo acumulado + heurística`. A função `get_next_astar` cuida de calcular isso e adicionar os próximos estados à fila.





In [51]:
def Astar(puzzle):
    puzzle = int("".join([str(x) for x in puzzle]).replace("0", "9"))
    GOAL = "123456789"
    queue = []
    heapq.heappush(queue, (0, 0, puzzle))
    visited = set()
    tree = {puzzle: -1}
    cost = {puzzle: 0}
    data = {
        "path_to_goal": [],
        "cost_of_path": 0,
        "nodes_expanded": 0,
        "fringe_size": 0,
        "max_fringe_size": 0,
        "search_depth": 0,
        "max_search_depth": 0,
        "running_time": time.time(),
        "max_ram_usage": 0
    }
    while queue:
        data["max_fringe_size"] = max(data["max_fringe_size"], len(queue))
        data["max_ram_usage"] = max(data["max_ram_usage"], get_ram_usage())
        _, depth, current = heapq.heappop(queue)
        if current in visited:
            continue
        visited.add(current)
        current = str(current)
        data["nodes_expanded"] += 1
        data["search_depth"] = max(data["search_depth"], depth)
        data["max_search_depth"] = max(data["max_search_depth"], depth)
        if current == GOAL:
            current = int(current)
            while tree[current] != -1:
                data["path_to_goal"].append(rebuild_path(str(tree[current]), str(current)))
                current = int(tree[current])
            data["path_to_goal"].reverse()
            data["cost_of_path"] = len(data["path_to_goal"])
            data["fringe_size"] = len(queue)
            data["running_time"] = time.time() - data["running_time"]
            return data
        i = current.index('9')
        if i % 3 != 2: # right
            get_next_astar(i, i + 1, current, tree, queue, cost, depth)
        if i % 3 != 0: # left
            get_next_astar(i, i - 1, current, tree, queue, cost, depth)
        if i < 6: # down
            get_next_astar(i, i + 3, current, tree, queue, cost, depth)
        if i > 2: # up
            get_next_astar(i, i - 3, current, tree, queue, cost, depth)
    data["fringe_size"] = len(queue)
    data["running_time"] = time.time() - data["running_time"]
    data["path_to_goal"] = "Sem solução"
    return data

## busca_gulosa(puzzle)
### Algoritmo de busca gulosa
O algoritmo de busca gulosa se diferencia do A* no momento onde são calculadas as distâncias, o A* utiliza a distância total, diferentemente do de busca gulosa, que calcula apenas a distância para o próximo estado

In [52]:
def busca_gulosa(puzzle):
    puzzle = int("".join([str(x) for x in puzzle]).replace("0", "9"))
    GOAL = "123456789"
    queue = []
    heapq.heappush(queue, (manhattan(str(puzzle)), 0, puzzle))
    visited = set()
    tree = {puzzle: -1}
    data = {
        "path_to_goal": [],
        "cost_of_path": 0,
        "nodes_expanded": 0,
        "fringe_size": 0,
        "max_fringe_size": 0,
        "search_depth": 0,
        "max_search_depth": 0,
        "running_time": time.time(),
        "max_ram_usage": 0
    }
    while queue:
        data["max_fringe_size"] = max(data["max_fringe_size"], len(queue))
        data["max_ram_usage"] = max(data["max_ram_usage"], get_ram_usage())
        _, depth, current = heapq.heappop(queue)
        if current in visited:
            continue
        visited.add(current)
        current = str(current)
        data["nodes_expanded"] += 1
        data["search_depth"] = max(data["search_depth"], depth)
        data["max_search_depth"] = max(data["max_search_depth"], depth)
        if current == GOAL:
            current = int(current)
            while tree[current] != -1:
                data["path_to_goal"].append(rebuild_path(str(tree[current]), str(current)))
                current = int(tree[current])
            data["path_to_goal"].reverse()
            data["cost_of_path"] = len(data["path_to_goal"])
            data["fringe_size"] = len(queue)
            data["running_time"] = time.time() - data["running_time"]
            return data
        i = current.index('9')
        if i % 3 != 2: # right
            get_next_gulosa(i, i + 1, current, tree, queue, depth)
        if i % 3 != 0: # left
            get_next_gulosa(i, i - 1, current, tree, queue, depth)
        if i < 6: # down
            get_next_gulosa(i, i + 3, current, tree, queue, depth)
        if i > 2: # up
            get_next_gulosa(i, i - 3, current, tree, queue, depth)
    data["fringe_size"] = len(queue)
    data["running_time"] = time.time() - data["running_time"]
    data["path_to_goal"] = "Sem solução"
    return data

# Exemplo de uso

In [ ]:
puzzle = (0, 8, 7, 6, 5, 4, 3, 2, 1)
print("A*")
for key, value in Astar(puzzle).items():
    print(f"{key}: {value}")
print("Busca Gulosa")
for key, value in busca_gulosa(puzzle).items():
    print(f"{key}: {value}")

## Análise
É possível observar que ambos os algoritmos chegaram na mesma solução, porém o A* obteve um desempenho 20x pior, tanto em relação ao número de nós expandido quanto em relação ao tempo, essa diferença se dá devido ao jeito como os caminhos são salvos na fila.

In [ ]:
puzzle = (1,2,3,4,5,7,6,0,8)
print("A*")
for key, value in Astar(puzzle).items():
    print(f"{key}: {value}")
print("Busca Gulosa")
for key, value in busca_gulosa(puzzle).items():
    print(f"{key}: {value}")

## Análise
Mesmo quando a solução não existe, o tempo médio de execução dos algoritmos é bem baixo, o que mostra uma superioridade em relação aos outros algoritmos testados nesse problema (BFS, DFS e IDFS)